In [1]:
import os
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', 100)

In [2]:
os.chdir('..')

In [3]:
!pwd

/Users/A.IVA/Documents/jupyter_notebooks/rest/coursera_and_blogs/time_series


In [4]:
from ts_utils.preprocess import *

### 1 Load data

In [5]:
data_dir = os.path.join('data', 'rossmann-store-sales')
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

df_store = pd.read_csv( os.path.join(data_dir, 'store.csv'))

df_train = pd.read_csv( os.path.join(data_dir, 'train.csv'), parse_dates=[2], low_memory=False)
df_test = pd.read_csv( os.path.join(data_dir, 'test.csv'), parse_dates=[3], date_parser=dateparse)

In [6]:
print(df_train.shape, df_test.shape, df_store.shape)
display(df_store.head(2))
display(df_train.head(2))
display(df_test.head(2))

(1017209, 9) (41088, 8) (1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0


### 2 Data preprocessing

##### 2.1 use transformers

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

In [8]:
# store open transformation

store_open_imputer = SimpleImputer(strategy='constant', fill_value=1, copy=True)
df_test['Open'] = store_open_imputer.fit_transform(df_test[['Open']]).reshape(-1)

In [9]:
# set all values of `Open=1` to `0` if `Customers==0`

store_open_trf = FunctionTransformer(lambda x: np.array([modify_store_open(*el) for el in x]).reshape(-1,1), 
                                     validate=False)

df_train['Open'] = store_open_trf.fit_transform(df_train[['Open', 'Customers']].values).reshape(-1)

In [10]:
# get promo date

promo_2_since_trf = FunctionTransformer(lambda x: np.array([get_promo_date(*el) for el in x]).reshape(-1,1), 
                                        validate=False)

promo_2_since_trf.fit(df_store[['Promo2SinceYear','Promo2SinceWeek']].values)
df_store['Promo2Since'] = promo_2_since_trf.transform(df_store[['Promo2SinceYear','Promo2SinceWeek']].values).reshape(-1)

df_store.drop(['Promo2SinceYear','Promo2SinceWeek'], inplace=True, axis=1)

In [11]:
# get promo months 

promo_2_months = FunctionTransformer(lambda x: np.array([get_promo_months(*el) for el in x]).reshape(-1,1), 
                                        validate=False)

promo_2_months.fit(df_store[['PromoInterval']].values)
df_store['PromoInterval'] = promo_2_months.transform(df_store[['PromoInterval']].values).reshape(-1)

In [12]:
# get date when the competition has appeared

competition_appear_trf = FunctionTransformer(lambda x: np.array([get_competition_date(*el) for el in x]).reshape(-1,1), 
                                             validate=False)

competition_appear_trf.fit(df_store[['CompetitionOpenSinceYear','CompetitionOpenSinceMonth']].values)
df_store['CompetitionSince'] = competition_appear_trf.transform(df_store[['CompetitionOpenSinceYear','CompetitionOpenSinceMonth']].values).reshape(-1)

df_store.drop(['CompetitionOpenSinceYear','CompetitionOpenSinceMonth'], inplace=True, axis=1)

In [13]:
# Competition distance TODO: later 
competition_distance_imputer = SimpleImputer(strategy='mean', copy=True)
competition_distance_imputer.fit(df_store[['CompetitionDistance']]) 
df_store['CompetitionDistance'] = competition_distance_imputer.transform(df_store[['CompetitionDistance']]).reshape(-1)

In [14]:
df_store['CompetitionDistance'] = df_store['CompetitionDistance'].astype(int)
df_store['CompetitionSince'] = pd.to_datetime(df_store['CompetitionSince'])
df_store['Promo2Since'] = pd.to_datetime(df_store['Promo2Since'])

df_store.set_index('Store', inplace=True)

In [15]:
display(df_store.head(3))
df_store.dtypes

,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Promo2Since,CompetitionSince
Store,,,,,,,
1,c,a,1270,0,[],NaT,2008-09-01
2,a,a,570,1,"[1, 4, 7, 10]",2010-04-01,2007-11-01
3,a,a,14130,1,"[1, 4, 7, 10]",2011-04-01,2006-12-01


StoreType                      object
Assortment                     object
CompetitionDistance             int64
Promo2                          int64
PromoInterval                  object
Promo2Since            datetime64[ns]
CompetitionSince       datetime64[ns]
dtype: object

In [16]:
# join train/test with store data (df_store)

df_train = df_train.join(df_store, on='Store')
df_test = df_test.join(df_store, on='Store')

In [17]:
display(df_train.head(3))
display(df_test.head(3))

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Promo2Since,CompetitionSince
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270,0,[],NaT,2008-09-01
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570,1,"[1, 4, 7, 10]",2010-04-01,2007-11-01
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130,1,"[1, 4, 7, 10]",2011-04-01,2006-12-01


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Promo2Since,CompetitionSince
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270,0,[],NaT,2008-09-01
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130,1,"[1, 4, 7, 10]",2011-04-01,2006-12-01
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000,0,[],NaT,2013-04-01


In [18]:
df_train['Assortment'].unique()

array(['a', 'c', 'b'], dtype=object)

In [19]:
df_train.dtypes

Store                           int64
DayOfWeek                       int64
Date                   datetime64[ns]
Sales                           int64
Customers                       int64
Open                            int64
Promo                           int64
StateHoliday                   object
SchoolHoliday                   int64
StoreType                      object
Assortment                     object
CompetitionDistance             int64
Promo2                          int64
PromoInterval                  object
Promo2Since            datetime64[ns]
CompetitionSince       datetime64[ns]
dtype: object

In [20]:
# time after last promotion (if exists)

dt_last_prom_trf = FunctionTransformer(lambda x: np.array([dt_from_last_promotion(*el) for el in x]).reshape(-1,1), 
                                        validate=False)

df_train['dtLastProm'] = dt_last_prom_trf.fit_transform(df_train[['Date','PromoInterval','Promo2Since']].values).reshape(-1)
df_test['dtLastProm'] = dt_last_prom_trf.fit_transform(df_test[['Date','PromoInterval','Promo2Since']].values).reshape(-1)

In [21]:
display(df_train.head(3))
display(df_test.head(3))

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Promo2Since,CompetitionSince,dtLastProm
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270,0,[],NaT,2008-09-01,None
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570,1,"[1, 4, 7, 10]",2010-04-01,2007-11-01,30
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130,1,"[1, 4, 7, 10]",2011-04-01,2006-12-01,30


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Promo2Since,CompetitionSince,dtLastProm
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270,0,[],NaT,2008-09-01,None
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130,1,"[1, 4, 7, 10]",2011-04-01,2006-12-01,78
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000,0,[],NaT,2013-04-01,None


In [22]:
df_train.to_csv(os.path.join(data_dir, 'train_trf.csv'), index=False)
df_test.to_csv(os.path.join(data_dir, 'test_trf.csv'), index=False)

In [23]:
data_dir = os.path.join('data', 'rossmann-store-sales')

df_train = pd.read_csv(os.path.join(data_dir, 'train_trf.csv'), low_memory=False)
df_test = pd.read_csv(os.path.join(data_dir, 'test_trf.csv'), low_memory=False)

In [24]:
from ast import literal_eval

for df in [df_train, df_test]:
    df['Date'] = pd.to_datetime(df['Date'])
    df['CompetitionSince'] = pd.to_datetime(df['CompetitionSince'])
    df['Promo2Since'] = pd.to_datetime(df['Promo2Since'])
    df['PromoInterval'] = df['PromoInterval'].apply(lambda x: literal_eval(x))

In [25]:
import seaborn as sns
import xgboost as xgb

In [26]:
date_cols = ['Date', 'Promo2Since', 'CompetitionSince']
non_date_cols = list(df_train.columns.values)
non_date_cols = [el for el in non_date_cols if el not in date_cols]

In [27]:
# this does not work !!!!!!!!!!!!!!!!!!!!!
df_train.loc[:, non_date_cols].fillna(-1, inplace=True)
df_test.loc[:, non_date_cols].fillna(-1, inplace=True)

df_train.loc[:, date_cols].fillna(pd.NaT, inplace=True)
df_test.loc[:, date_cols].fillna(pd.NaT, inplace=True)

/anaconda3/envs/ts/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [30]:
display(df_train.isnull().sum(axis = 0))
display(df_test.isnull().sum(axis = 0))

Store                       0
DayOfWeek                   0
Date                        0
Sales                       0
Customers                   0
Open                        0
Promo                       0
StateHoliday                0
SchoolHoliday               0
StoreType                   0
Assortment                  0
CompetitionDistance         0
Promo2                      0
PromoInterval               0
Promo2Since            508031
CompetitionSince       323348
dtLastProm             580269
dtype: int64

Id                         0
Store                      0
DayOfWeek                  0
Date                       0
Open                       0
Promo                      0
StateHoliday               0
SchoolHoliday              0
StoreType                  0
Assortment                 0
CompetitionDistance        0
Promo2                     0
PromoInterval              0
Promo2Since            17232
CompetitionSince       15216
dtLastProm             17232
dtype: int64

In [31]:
# train/validation split

There are not stores in the `test` set that are not present in the `train` set.

In [32]:
print(len(list(set(df_train['Store'].values))), len(list(set(df_test['Store'].values))))
print(min(list(set(df_train['Store'].values))), max(list(set(df_train['Store'].values))), min(list(set(df_test['Store'].values))), max(list(set(df_test['Store'].values))))

1115 856
1 1115 1 1115


In [33]:
df_val = df_train[df_train['Date']>='2015-06-01'].copy(deep=True)
df_train = df_train[df_train['Date']<'2015-06-01'].copy(deep=True)
df_val.shape, df_train.shape

((68015, 17), (949194, 17))

In [34]:
# only use data of Sales>0 and Open is 1
df_val = df_val[(df_val['Open'] != 0) & (df_val['Sales'] != 0)]
df_train = df_train[(df_train['Open'] != 0) & (df_train['Sales'] != 0)]

In [35]:
df_train.head(2)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Promo2Since,CompetitionSince,dtLastProm
68099,85,7,2015-05-31,12040,1647,1,0,0,0,b,a,1870,0,[],NaT,2011-10-01,NaN
68136,122,7,2015-05-31,2737,353,1,0,0,0,a,c,58260,0,[],NaT,2013-04-01,NaN


In [36]:
# One hot encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

weekday_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
state_holiday_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
school_holiday_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
store_type_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
assortment_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

weekday_encoder.fit(df_train[['DayOfWeek']].values)
state_holiday_encoder.fit(df_train[['StateHoliday']].values)
school_holiday_encoder.fit(df_train[['SchoolHoliday']].values)
store_type_encoder.fit(df_train[['StoreType']].values)
assortment_encoder.fit(df_train[['Assortment']].values)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=False)

In [244]:
# df_train['StateHoliday'].unique(), df_val['StateHoliday'].unique()

In [245]:
# col = 'StateHoliday'
# encoder = state_holiday_encoder

# pd.DataFrame(encoder.transform(df_val[[col]].values),
#              columns=list(map(lambda x: '{0:s}_{1:s}'.format(col, x), encoder.get_feature_names())))[:2]

# pd.DataFrame(encoder.transform(df_train[[col]].values),
#              columns=list(map(lambda x: '{0:s}_{1:s}'.format(col, x), encoder.get_feature_names())))[:2]

In [37]:
encoders = [ weekday_encoder, state_holiday_encoder, school_holiday_encoder, store_type_encoder, assortment_encoder]
columns = ['DayOfWeek','StateHoliday','SchoolHoliday','StoreType','Assortment']
    
def update_df(df):

    for col, encoder in zip(columns, encoders):
        
        new_columns=list(map(lambda x: '{0:s}_{1:s}'.format(col, x), encoder.get_feature_names()))
    
        df[new_columns] = pd.DataFrame(encoder.transform(df[[col]].values), index=df.index)
                                   
        df.drop([col], inplace=True, axis=1)

    return None

In [38]:
update_df(df_train)
update_df(df_val)

In [39]:
df_train.drop(['PromoInterval'], inplace=True, axis=1)
df_val.drop(['PromoInterval'], inplace=True, axis=1)

In [40]:
display(df_train.head(2))
display(df_val.head(2))
display(df_test.head(2))

,Store,Date,Sales,Customers,Open,Promo,CompetitionDistance,Promo2,Promo2Since,CompetitionSince,dtLastProm,DayOfWeek_x0_1,DayOfWeek_x0_2,DayOfWeek_x0_3,DayOfWeek_x0_4,DayOfWeek_x0_5,DayOfWeek_x0_6,DayOfWeek_x0_7,StateHoliday_x0_0,StateHoliday_x0_a,StateHoliday_x0_b,StateHoliday_x0_c,SchoolHoliday_x0_0,SchoolHoliday_x0_1,StoreType_x0_a,StoreType_x0_b,StoreType_x0_c,StoreType_x0_d,Assortment_x0_a,Assortment_x0_b,Assortment_x0_c
68099,85,2015-05-31,12040,1647,1,0,1870,0,NaT,2011-10-01,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
68136,122,2015-05-31,2737,353,1,0,58260,0,NaT,2013-04-01,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


,Store,Date,Sales,Customers,Open,Promo,CompetitionDistance,Promo2,Promo2Since,CompetitionSince,dtLastProm,DayOfWeek_x0_1,DayOfWeek_x0_2,DayOfWeek_x0_3,DayOfWeek_x0_4,DayOfWeek_x0_5,DayOfWeek_x0_6,DayOfWeek_x0_7,StateHoliday_x0_0,StateHoliday_x0_a,StateHoliday_x0_b,StateHoliday_x0_c,SchoolHoliday_x0_0,SchoolHoliday_x0_1,StoreType_x0_a,StoreType_x0_b,StoreType_x0_c,StoreType_x0_d,Assortment_x0_a,Assortment_x0_b,Assortment_x0_c
0,1,2015-07-31,5263,555,1,1,1270,0,NaT,2008-09-01,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,2,2015-07-31,6064,625,1,1,570,1,2010-04-01,2007-11-01,30.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Promo2Since,CompetitionSince,dtLastProm
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270,0,[],NaT,2008-09-01,NaN
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130,1,"[1, 4, 7, 10]",2011-04-01,2006-12-01,78.0


In [41]:
Y_train = np.log1p(df_train['Sales'])
Y_val = np.log1p(df_val['Sales'])

X_train = df_train.drop(['Sales'], inplace=False, axis=1)
X_val = df_val.drop(['Sales'], inplace=False, axis=1)

In [42]:
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

In [43]:
X_train.drop(['Date', 'Promo2Since', 'CompetitionSince'], inplace=True, axis=1)
X_val.drop(['Date', 'Promo2Since', 'CompetitionSince'], inplace=True, axis=1)

In [46]:
X_train.drop(['Customers'], inplace=True, axis=1)
X_val.drop(['Customers'], inplace=True, axis=1)

In [49]:
X_train.drop(['Store'], inplace=True, axis=1)
X_val.drop(['Store'], inplace=True, axis=1)

In [50]:
X_train.head(2)

,Open,Promo,CompetitionDistance,Promo2,dtLastProm,DayOfWeek_x0_1,DayOfWeek_x0_2,DayOfWeek_x0_3,DayOfWeek_x0_4,DayOfWeek_x0_5,DayOfWeek_x0_6,DayOfWeek_x0_7,StateHoliday_x0_0,StateHoliday_x0_a,StateHoliday_x0_b,StateHoliday_x0_c,SchoolHoliday_x0_0,SchoolHoliday_x0_1,StoreType_x0_a,StoreType_x0_b,StoreType_x0_c,StoreType_x0_d,Assortment_x0_a,Assortment_x0_b,Assortment_x0_c
68099,1,0,1870,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
68136,1,0,58260,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [51]:
from time import time

params = {
    "objective": "reg:linear",
    "booster" : "gbtree",
    "eta": 0.03,
    "max_depth": 10,
    "subsample": 0.9,
    "colsample_bytree": 0.7,
    "silent": 1,
    "seed": 10
    }

num_boost_round = 6000

dtrain = xgb.DMatrix(X_train, Y_train)
dvalid = xgb.DMatrix(X_val, Y_val)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]


print("Train a XGBoost model")
start = time()
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, 
      early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)
end = time()
print('Training time is {:2f} s.'.format(end-start))

Train a XGBoost model
[0]	train-rmse:8.01877	eval-rmse:8.05388	train-rmspe:0.999808	eval-rmspe:0.999816
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:7.77876	eval-rmse:7.8138	train-rmspe:0.999709	eval-rmspe:0.999721
[2]	train-rmse:7.54591	eval-rmse:7.5808	train-rmspe:0.999589	eval-rmspe:0.999605
[3]	train-rmse:7.3201	eval-rmse:7.35493	train-rmspe:0.999442	eval-rmspe:0.999464
[4]	train-rmse:7.10102	eval-rmse:7.13569	train-rmspe:0.999264	eval-rmspe:0.999293
[5]	train-rmse:6.88858	eval-rmse:6.92314	train-rmspe:0.99905	eval-rmspe:0.999088
[6]	train-rmse:6.6825	eval-rmse:6.71706	train-rmspe:0.998794	eval-rmspe:0.998842
[7]	train-rmse:6.48258	eval-rmse:6.51703	train-rmspe:0.998491	eval-rmspe:0.998552
[8]	train-rmse:6.28875	eval-rmse:6.3231	train-rmspe:0.998133	eval-rmspe:0.998208
[9]	train-rmse:6.10069	eval-rmse:6.13486	train-rmspe:0.997713	eval-rmspe:0.997805
[10]	train-rmse:5.

[97]	train-rmse:0.5145	eval-rmse:0.529824	train-rmspe:0.391064	eval-rmspe:0.388154
[98]	train-rmse:0.504121	eval-rmse:0.518994	train-rmspe:0.385961	eval-rmspe:0.382058
[99]	train-rmse:0.494169	eval-rmse:0.508557	train-rmspe:0.38105	eval-rmspe:0.376147
[100]	train-rmse:0.484761	eval-rmse:0.498554	train-rmspe:0.376456	eval-rmspe:0.370461
[101]	train-rmse:0.475317	eval-rmse:0.488651	train-rmspe:0.371848	eval-rmspe:0.364863
[102]	train-rmse:0.466374	eval-rmse:0.47915	train-rmspe:0.367511	eval-rmspe:0.359443
[103]	train-rmse:0.457806	eval-rmse:0.470068	train-rmspe:0.363417	eval-rmspe:0.354271
[104]	train-rmse:0.449597	eval-rmse:0.461376	train-rmspe:0.359531	eval-rmspe:0.349301
[105]	train-rmse:0.441928	eval-rmse:0.45321	train-rmspe:0.356007	eval-rmspe:0.344652
[106]	train-rmse:0.434284	eval-rmse:0.445068	train-rmspe:0.352467	eval-rmspe:0.339999
[107]	train-rmse:0.427296	eval-rmse:0.4376	train-rmspe:0.349354	eval-rmspe:0.335758
[108]	train-rmse:0.420187	eval-rmse:0.430045	train-rmspe:0.34615

[193]	train-rmse:0.26445	eval-rmse:0.251357	train-rmspe:0.312622	eval-rmspe:0.250902
[194]	train-rmse:0.264005	eval-rmse:0.250861	train-rmspe:0.312463	eval-rmspe:0.250599
[195]	train-rmse:0.263854	eval-rmse:0.250715	train-rmspe:0.312606	eval-rmspe:0.250734
[196]	train-rmse:0.263423	eval-rmse:0.250235	train-rmspe:0.312445	eval-rmspe:0.250445
[197]	train-rmse:0.263293	eval-rmse:0.250073	train-rmspe:0.312596	eval-rmspe:0.250563
[198]	train-rmse:0.262836	eval-rmse:0.249605	train-rmspe:0.312396	eval-rmspe:0.250301
[199]	train-rmse:0.262149	eval-rmse:0.248833	train-rmspe:0.311819	eval-rmspe:0.249664
[200]	train-rmse:0.261292	eval-rmse:0.247904	train-rmspe:0.311175	eval-rmspe:0.248927
[201]	train-rmse:0.260992	eval-rmse:0.247562	train-rmspe:0.311109	eval-rmspe:0.248773
[202]	train-rmse:0.259998	eval-rmse:0.246415	train-rmspe:0.310324	eval-rmspe:0.247738
[203]	train-rmse:0.259559	eval-rmse:0.245933	train-rmspe:0.310114	eval-rmspe:0.247403
[204]	train-rmse:0.259046	eval-rmse:0.245375	train-rmsp

[289]	train-rmse:0.239596	eval-rmse:0.226044	train-rmspe:0.296581	eval-rmspe:0.232958
[290]	train-rmse:0.239588	eval-rmse:0.226044	train-rmspe:0.296583	eval-rmspe:0.232961
[291]	train-rmse:0.239239	eval-rmse:0.22573	train-rmspe:0.296293	eval-rmspe:0.232676
[292]	train-rmse:0.238767	eval-rmse:0.225253	train-rmspe:0.295861	eval-rmspe:0.232244
[293]	train-rmse:0.238377	eval-rmse:0.224892	train-rmspe:0.295517	eval-rmspe:0.2319
[294]	train-rmse:0.238261	eval-rmse:0.224783	train-rmspe:0.295444	eval-rmspe:0.231802
[295]	train-rmse:0.238213	eval-rmse:0.224793	train-rmspe:0.295413	eval-rmspe:0.23183
[296]	train-rmse:0.237986	eval-rmse:0.224572	train-rmspe:0.295176	eval-rmspe:0.231629
[297]	train-rmse:0.237683	eval-rmse:0.224273	train-rmspe:0.294869	eval-rmspe:0.231338
[298]	train-rmse:0.237654	eval-rmse:0.224274	train-rmspe:0.294854	eval-rmspe:0.231342
[299]	train-rmse:0.237308	eval-rmse:0.223956	train-rmspe:0.294504	eval-rmspe:0.23103
[300]	train-rmse:0.237299	eval-rmse:0.223956	train-rmspe:0.

[385]	train-rmse:0.225524	eval-rmse:0.213781	train-rmspe:0.283142	eval-rmspe:0.221212
[386]	train-rmse:0.225293	eval-rmse:0.213557	train-rmspe:0.282893	eval-rmspe:0.220969
[387]	train-rmse:0.224858	eval-rmse:0.213119	train-rmspe:0.282504	eval-rmspe:0.220604
[388]	train-rmse:0.22472	eval-rmse:0.213019	train-rmspe:0.282385	eval-rmspe:0.220503
[389]	train-rmse:0.224404	eval-rmse:0.212705	train-rmspe:0.282077	eval-rmspe:0.220158
[390]	train-rmse:0.224371	eval-rmse:0.212707	train-rmspe:0.282051	eval-rmspe:0.220169
[391]	train-rmse:0.224244	eval-rmse:0.212573	train-rmspe:0.281943	eval-rmspe:0.220058
[392]	train-rmse:0.224108	eval-rmse:0.21245	train-rmspe:0.281827	eval-rmspe:0.219934
[393]	train-rmse:0.223989	eval-rmse:0.212374	train-rmspe:0.281713	eval-rmspe:0.219856
[394]	train-rmse:0.223663	eval-rmse:0.212041	train-rmspe:0.281398	eval-rmspe:0.219506
[395]	train-rmse:0.2236	eval-rmse:0.211992	train-rmspe:0.281344	eval-rmspe:0.219448
[396]	train-rmse:0.22343	eval-rmse:0.21185	train-rmspe:0.2

...

[2577]	train-rmse:0.17447	eval-rmse:0.175319	train-rmspe:0.237019	eval-rmspe:0.184167
[2578]	train-rmse:0.174466	eval-rmse:0.175317	train-rmspe:0.237015	eval-rmspe:0.184163
[2579]	train-rmse:0.174464	eval-rmse:0.175315	train-rmspe:0.237012	eval-rmspe:0.184162
[2580]	train-rmse:0.174464	eval-rmse:0.175317	train-rmspe:0.237012	eval-rmspe:0.184165
[2581]	train-rmse:0.174462	eval-rmse:0.175315	train-rmspe:0.237012	eval-rmspe:0.184163
[2582]	train-rmse:0.174462	eval-rmse:0.175316	train-rmspe:0.237011	eval-rmspe:0.184163
[2583]	train-rmse:0.174459	eval-rmse:0.175312	train-rmspe:0.237011	eval-rmspe:0.184163
[2584]	train-rmse:0.17445	eval-rmse:0.175306	train-rmspe:0.237002	eval-rmspe:0.184155
[2585]	train-rmse:0.174443	eval-rmse:0.1753	train-rmspe:0.236997	eval-rmspe:0.184149
[2586]	train-rmse:0.174442	eval-rmse:0.175306	train-rmspe:0.236998	eval-rmspe:0.184154
[2587]	train-rmse:0.174429	eval-rmse:0.1753	train-rmspe:0.236981	eval-rmspe:0.18415
[2588]	train-rmse:0.174428	eval-rmse:0.175299	trai

[2672]	train-rmse:0.174149	eval-rmse:0.175233	train-rmspe:0.236679	eval-rmspe:0.184058
[2673]	train-rmse:0.174142	eval-rmse:0.175231	train-rmspe:0.236672	eval-rmspe:0.184052
[2674]	train-rmse:0.174142	eval-rmse:0.17523	train-rmspe:0.236671	eval-rmspe:0.184052
[2675]	train-rmse:0.174133	eval-rmse:0.175224	train-rmspe:0.236664	eval-rmspe:0.184045
[2676]	train-rmse:0.174132	eval-rmse:0.175231	train-rmspe:0.23666	eval-rmspe:0.184053
[2677]	train-rmse:0.17413	eval-rmse:0.175231	train-rmspe:0.236653	eval-rmspe:0.184052
[2678]	train-rmse:0.174128	eval-rmse:0.175229	train-rmspe:0.236649	eval-rmspe:0.184049
[2679]	train-rmse:0.174128	eval-rmse:0.175229	train-rmspe:0.236653	eval-rmspe:0.18405
[2680]	train-rmse:0.174128	eval-rmse:0.175229	train-rmspe:0.236654	eval-rmspe:0.18405
[2681]	train-rmse:0.174127	eval-rmse:0.175228	train-rmspe:0.236651	eval-rmspe:0.184048
[2682]	train-rmse:0.174124	eval-rmse:0.175228	train-rmspe:0.236644	eval-rmspe:0.184047
[2683]	train-rmse:0.174123	eval-rmse:0.175226	tr

[2767]	train-rmse:0.173799	eval-rmse:0.175123	train-rmspe:0.236278	eval-rmspe:0.18389
[2768]	train-rmse:0.173796	eval-rmse:0.175121	train-rmspe:0.236274	eval-rmspe:0.183888
[2769]	train-rmse:0.173789	eval-rmse:0.175117	train-rmspe:0.236268	eval-rmspe:0.183879
[2770]	train-rmse:0.173789	eval-rmse:0.175122	train-rmspe:0.236268	eval-rmspe:0.183885
[2771]	train-rmse:0.173784	eval-rmse:0.175117	train-rmspe:0.236263	eval-rmspe:0.183879
[2772]	train-rmse:0.173779	eval-rmse:0.175114	train-rmspe:0.23626	eval-rmspe:0.183875
[2773]	train-rmse:0.173777	eval-rmse:0.175116	train-rmspe:0.236259	eval-rmspe:0.183877
[2774]	train-rmse:0.173772	eval-rmse:0.175114	train-rmspe:0.236252	eval-rmspe:0.183868
[2775]	train-rmse:0.173762	eval-rmse:0.175111	train-rmspe:0.236241	eval-rmspe:0.183863
[2776]	train-rmse:0.173762	eval-rmse:0.175112	train-rmspe:0.23624	eval-rmspe:0.183864
[2777]	train-rmse:0.173756	eval-rmse:0.175107	train-rmspe:0.236232	eval-rmspe:0.183849
[2778]	train-rmse:0.173755	eval-rmse:0.175111	

[2862]	train-rmse:0.173522	eval-rmse:0.17506	train-rmspe:0.235957	eval-rmspe:0.183762
[2863]	train-rmse:0.173521	eval-rmse:0.175064	train-rmspe:0.235959	eval-rmspe:0.183764
[2864]	train-rmse:0.173515	eval-rmse:0.175059	train-rmspe:0.235951	eval-rmspe:0.183762
[2865]	train-rmse:0.173511	eval-rmse:0.175056	train-rmspe:0.235949	eval-rmspe:0.183756
[2866]	train-rmse:0.17351	eval-rmse:0.175064	train-rmspe:0.235948	eval-rmspe:0.183767
[2867]	train-rmse:0.173509	eval-rmse:0.175062	train-rmspe:0.235947	eval-rmspe:0.183765
[2868]	train-rmse:0.173504	eval-rmse:0.175062	train-rmspe:0.235946	eval-rmspe:0.183762
[2869]	train-rmse:0.1735	eval-rmse:0.17506	train-rmspe:0.235942	eval-rmspe:0.18376
[2870]	train-rmse:0.173493	eval-rmse:0.175058	train-rmspe:0.235932	eval-rmspe:0.183754
[2871]	train-rmse:0.173492	eval-rmse:0.175056	train-rmspe:0.23593	eval-rmspe:0.18375
[2872]	train-rmse:0.173489	eval-rmse:0.175056	train-rmspe:0.235917	eval-rmspe:0.183749
[2873]	train-rmse:0.173488	eval-rmse:0.175056	train

KeyboardInterrupt: 

In [ ]:
# 0.07 -> 0.14 (no customers) -> (no customers, no stores) -> 0.18

In [ ]:
print("validating")
X_val.sort_index(inplace=True) 
Y_val.sort_index(inplace=True) 
yhat = gbm.predict(xgb.DMatrix(X_val))
error = rmspe(np.expm1(Y_val), np.expm1(yhat))

print('RMSPE: {:.6f}'.format(error))